In [5]:
import face_recognition
import cv2
import numpy as np
import requests
import base64
import paramiko

In [6]:
# parameter
client_name = 'Kay'
gesture_name = 'Ok'
timeout = 1
stay_time = 10

remote_ip = "10.10.21.177"
remote_name = 'pi'
remote_password = ''
command = 'python ...' #FIXME

In [7]:
video_capture = cv2.VideoCapture(0)

# video_capture = cv2.VideoCapture()
# video_capture.open('rtmp://47.93.218.97:1935/live')

host = 'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id=brRx6PAaQLbN8rQuAt3AwAA5&client_secret=3p6kz6kdUfZ1IrsNSvT4jq2LT384ruBj'
request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v1/gesture"

process_this_frame = True

In [8]:
known_face_names = [
    "Obama",
    "Kay",
    "Wendi",
    "Muyun",
    "Zhimeng",
#     "Maggie"
]
known_face_encodings = []

for name in known_face_names:
    known_face_encodings.append(face_recognition.face_encodings(face_recognition.load_image_file("./faces/" + name + ".jpeg"))[0])
    
# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
percentages = []
process_this_frame = True

face_match = 0
hand_match = 0
t = 0

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()
    
    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            dis_sum = sum(face_distances)
            
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
                
            face_names.append(name)
            percentages.append([1 - i / dis_sum for i in face_distances][best_match_index])
            
        response = requests.get(host)

        # gesture recognition with baidu ai
        img_str = cv2.imencode('.jpg', frame)[1].tostring()
        img = base64.b64encode(img_str)

        response = requests.get(host)
        
        params = {"image":img}
        if response:
            access_token = response.json()['access_token']
        else:
            printf('Access Deny.')
        request_url = request_url + "?access_token=" + access_token
        headers = {'content-type': 'application/x-www-form-urlencoded'}
        response = requests.post(request_url, data=params, headers=headers)
        if response:
            results = response.json()['result']
        else:
            print('ERROR: response failed.')
            
    process_this_frame = not process_this_frame
    
    # ------------------Control------------------
    if hand_match > 0:
        hand_match -= 1
    for r in results:
        if r['classname'] == gesture_name:
            hand_match = timeout
    
    if face_match > 0:
        face_match -= 1
    if name in face_names:
        if name == client_name:
            face_match = timeout
    # ------------------Control------------------
    
    # ---------------Visualization---------------
    # Display the hand results
    for r in results:
        name = r['classname']
        left = r['left']
        right = r['left'] + r['width']
        top = r['top']
        bottom = r['top'] + r['height']
        p = r['probability']
        
        if name == 'Face':
            continue
        
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name + " " + str(p)[:5], (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the face results
    for (top, right, bottom, left), name, percentage in zip(face_locations, face_names, percentages):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name + " " + str(percentage)[:5], (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.rectangle(frame, (0, 0), (200, 35), (0, 0, 255), cv2.FILLED)
    cv2.putText(frame, 'F: ' + str(face_match) + ' H: ' + str(hand_match) + " ", (6, 25), font, 1.0, (255, 255, 255), 1)
    
    if t > 0:
        t -= 1
        cv2.rectangle(frame, (0, 35), (200, 70), (0, 255, 0), cv2.FILLED)
        cv2.putText(frame, 'Approved', (6, 62), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)
    # ---------------Visualization---------------
    
    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    if face_match > 0 and hand_match > 0:
        # approved!
        print("Approved")
        t = stay_time
#         requests.get('http://192.168.43.207/pin?a=on')
        
    if t == 1:
        break
    
# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

Approved
Approved
Approved
Approved


KeyboardInterrupt: 